In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:

  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [6]:
X.shape, Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [7]:
# C is the embedding vector
C = torch.randn((27, 2))
C.dtype

torch.float32

In [8]:
C[5]

tensor([-0.3234,  0.7327])

In [9]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.3234,  0.7327])

In [10]:
# how to embed all the 32, 3 input
C[X].shape

torch.Size([32, 3, 2])

In [11]:
C[1]

tensor([0.9393, 0.8578])

In [12]:
X[13, 2]

tensor(1)

In [13]:
C[X][13, 2]

tensor([0.9393, 0.8578])

In [14]:
# Embedding
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [16]:
# we have 2 dimensional embedding and 3 of them, so input to w2 is 6 and arbitrarily lets take 100 neurons
w1 = torch.randn((6, 100))
b1 = torch.randn((100, ))

# emb: [32, 3, 2], w1: [6, 100]
# how to transform the embedding to match the dimensions of w1 - concatenation
emb[:, 0, :].shape

torch.Size([32, 2])

In [21]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

But using cat is not scalable as we are directly indexing, so instead we use unbind.

In [20]:
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [23]:
emb.view(32, 6)

tensor([[-0.0165, -0.5596, -0.0165, -0.5596, -0.0165, -0.5596],
        [-0.0165, -0.5596, -0.0165, -0.5596, -0.3234,  0.7327],
        [-0.0165, -0.5596, -0.3234,  0.7327,  0.5824,  0.3238],
        [-0.3234,  0.7327,  0.5824,  0.3238,  0.5824,  0.3238],
        [ 0.5824,  0.3238,  0.5824,  0.3238,  0.9393,  0.8578],
        [-0.0165, -0.5596, -0.0165, -0.5596, -0.0165, -0.5596],
        [-0.0165, -0.5596, -0.0165, -0.5596,  0.3087, -0.7848],
        [-0.0165, -0.5596,  0.3087, -0.7848,  0.5836,  1.1228],
        [ 0.3087, -0.7848,  0.5836,  1.1228,  1.1142,  1.2820],
        [ 0.5836,  1.1228,  1.1142,  1.2820,  2.2040,  0.6020],
        [ 1.1142,  1.2820,  2.2040,  0.6020,  1.1142,  1.2820],
        [ 2.2040,  0.6020,  1.1142,  1.2820,  0.9393,  0.8578],
        [-0.0165, -0.5596, -0.0165, -0.5596, -0.0165, -0.5596],
        [-0.0165, -0.5596, -0.0165, -0.5596,  0.9393,  0.8578],
        [-0.0165, -0.5596,  0.9393,  0.8578,  2.2040,  0.6020],
        [ 0.9393,  0.8578,  2.2040,  0.6

In [25]:
emb.shape[0]

32

In [26]:
h = torch.tanh(emb.view(-1, 6) @ w1 + b1) # -1 is equivalent to emb.shape[0].

In [27]:
h

tensor([[-0.5498, -0.2635, -0.0706,  ..., -0.6777,  0.9431,  0.9317],
        [-0.8889,  0.6424,  0.3533,  ..., -0.6743,  0.8973, -0.2786],
        [-0.6792, -0.2250,  0.2046,  ..., -0.5257,  0.3966,  0.8615],
        ...,
        [ 0.7563, -0.1268, -0.9941,  ..., -0.9995,  0.2687,  1.0000],
        [-0.6234, -0.9678, -0.9981,  ...,  0.9369, -0.9882,  0.9969],
        [-0.9949, -0.9925, -0.8717,  ..., -0.3587, -0.9936,  0.9996]])

In [28]:
w2 = torch.randn((100, 27))
b2 = torch.randn(27)

logits = h @ w2 + b2
logits.shape

torch.Size([32, 27])

In [29]:
counts = logits.exp()
prob = counts / (counts.sum(1, keepdims=True))
prob[0].sum()

tensor(1.)

In [30]:
prob[torch.arange(32), Y]

tensor([8.6490e-09, 3.0010e-05, 6.3851e-03, 9.3427e-01, 1.5837e-09, 6.6511e-10,
        3.2713e-06, 1.5585e-09, 1.1825e-05, 3.7837e-09, 9.9651e-01, 9.9945e-09,
        3.8487e-06, 3.1838e-04, 9.4444e-01, 3.5117e-09, 8.8450e-07, 1.1509e-10,
        1.5400e-01, 1.2306e-09, 9.5858e-12, 2.6757e-12, 7.1917e-11, 8.9618e-01,
        9.4018e-09, 2.1142e-04, 7.1474e-10, 2.1711e-10, 2.8891e-03, 5.9484e-11,
        3.6551e-01, 8.0932e-10])

In [31]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(14.1842)

In [32]:
l = F.cross_entropy(logits, Y) # much more efficient than writing our own code, as new tenors are not created in memory.
l

tensor(14.1842)